# 06. 추론 (Inference)
### 학습된 모델로 새 영상 강조 구간 검출


In [ ]:
import os
import cv2
import torch
import librosa
import numpy as np
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# =========================================================
# 1. 설정 (학습 코드와 동일하게 맞춤)
# =========================================================
SEGMENT_DURATION = 1.0
OVERLAP = 0.5
RESIZE_HW = (112, 112)
CLIP_LEN = 16
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 오디오 설정
AUDIO_CONFIG = {
    'sample_rate': 16000,
    'n_mels': 80,
    'n_fft': 1024,
    'hop_length': 512,
    'window_size': 50
}

# 퓨전 모델 설정 (학습 시 사용한 FUSION_CONFIG)
FUSION_CONFIG = {
    "gesture_dim": 512,
    "audio_dim": 128,
    "text_dim": 1024,
    "hidden_dim": 256,
}

# 경로 설정 (사용자 환경에 맞게 수정 필요)
MODEL_DIR = "/home/stu/ai_project/모델집합"
GESTURE_MODEL_PATH = os.path.join(MODEL_DIR, "gesture_model.pt")
AUDIO_MODEL_PATH = os.path.join(MODEL_DIR, "best_bi_lstm.pth")
FUSION_MODEL_PATH = os.path.join(MODEL_DIR, "fusion_model_gate.pt")

# =========================================================
# 2. 클래스 정의 (학습 코드 복사)
# =========================================================

class AudioFeatureExtractorInferenceStyle:
    """학습 코드와 100% 동일한 오디오 전처리 클래스"""
    def __init__(self, config=None):
        if config is None: config = AUDIO_CONFIG
        self.sr = config.get('sample_rate', 16000)
        self.n_mels = config.get('n_mels', 80)
        self.n_fft = config.get('n_fft', 1024)
        self.hop_length = config.get('hop_length', 512)
        self.window_size = config.get('window_size', 50)
    
    def process_full_audio(self, y_audio):
        # 1. Waveform Normalization
        y_audio = librosa.util.normalize(y_audio)
        
        # 2. Mel-Spectrogram
        mel = librosa.feature.melspectrogram(
            y=y_audio, sr=self.sr, n_mels=self.n_mels,
            n_fft=self.n_fft, hop_length=self.hop_length
        )
        features = librosa.power_to_db(mel, ref=np.max).T 
        
        # 3. Global Scaling (파일 전체 기준)
        scaler = StandardScaler()
        try:
            features_norm = scaler.fit_transform(features)
        except ValueError:
            return np.zeros((0, self.n_mels), dtype=np.float32)

        return features_norm.astype(np.float32)

class GatedFusion(nn.Module):
    """학습된 Fusion 모델 구조"""
    def __init__(self, gesture_dim, audio_dim, text_dim, hidden_dim, num_classes=1, use_text=True, dropout=0.5):
        super(GatedFusion, self).__init__()
        self.use_text = use_text
        
        self.g_net = nn.Sequential(
            nn.Linear(gesture_dim, hidden_dim), nn.ReLU(),
            nn.BatchNorm1d(hidden_dim), nn.Dropout(p=dropout)
        )
        self.a_net = nn.Sequential(
            nn.Linear(audio_dim, hidden_dim), nn.ReLU(),
            nn.BatchNorm1d(hidden_dim), nn.Dropout(p=dropout)
        )
        
        if use_text:
            self.t_net = nn.Sequential(
                nn.Linear(text_dim, hidden_dim), nn.ReLU(),
                nn.BatchNorm1d(hidden_dim), nn.Dropout(p=dropout)
            )
            gate_input_dim = hidden_dim * 3
        else:
            gate_input_dim = hidden_dim * 2

        self.gate_net = nn.Sequential(
            nn.Linear(gate_input_dim, hidden_dim), nn.Tanh(),
            nn.Linear(hidden_dim, 3 if use_text else 2),
            nn.Softmax(dim=1)
        )

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2), nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )

    def forward(self, gesture, audio, text=None):
        h_g = self.g_net(gesture)
        h_a = self.a_net(audio)
        features = [h_g, h_a]
        
        if self.use_text and text is not None:
            h_t = self.t_net(text)
            features.append(h_t)
            
        concat_feat = torch.cat(features, dim=1)
        gates = self.gate_net(concat_feat)
        
        h_fused = gates[:, 0:1] * h_g + gates[:, 1:2] * h_a
        if self.use_text and text is not None:
            h_fused += gates[:, 2:3] * h_t
            
        return self.classifier(h_fused)

# =========================================================
# 3. 모델 로드 및 전처리 함수
# =========================================================

def load_all_models():
    """모든 서브 모델과 퓨전 모델을 로드합니다."""
    # 1. Utils에서 서브 모델 로더 가져오기 (사용자 환경의 utils.py 가정)
    try:
        from utils import load_gesture_model, load_audio_model, create_text_model
    except ImportError:
        print("⚠️ utils.py를 찾을 수 없습니다. 모델 로드 함수를 직접 정의해야 합니다.")
        return None, None, None, None

    print("🔧 모델 로드 중...")
    
    # Gesture & Audio Model Load
    g_model = load_gesture_model(GESTURE_MODEL_PATH, DEVICE)
    a_model = load_audio_model(AUDIO_MODEL_PATH, DEVICE)
    t_model = create_text_model().to(DEVICE) # Text Model (RoBERTa 등)

    # Fusion Model Load
    f_model = GatedFusion(
        gesture_dim=FUSION_CONFIG['gesture_dim'],
        audio_dim=FUSION_CONFIG['audio_dim'],
        text_dim=FUSION_CONFIG['text_dim'],
        hidden_dim=FUSION_CONFIG['hidden_dim'],
        use_text=True
    ).to(DEVICE)
    
    # 가중치 로드
    if os.path.exists(FUSION_MODEL_PATH):
        f_model.load_state_dict(torch.load(FUSION_MODEL_PATH, map_location=DEVICE))
        print("✅ Fusion 모델 가중치 로드 완료")
    else:
        print(f"❌ Fusion 모델 파일 없음: {FUSION_MODEL_PATH}")

    g_model.eval()
    a_model.eval()
    t_model.eval()
    f_model.eval()
    
    return g_model, a_model, t_model, f_model

def preprocess_new_video(video_path):
    """
    새로운 비디오에 대해 학습 코드와 동일한 전처리를 수행합니다.
    Video: Resize -> Normalize
    Audio: Scaler -> Slice
    Text: (Optional) Placeholder or Embedding extraction
    """
    # 1. 비디오 세그먼트 생성
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps if fps > 0 else 0
    
    step = SEGMENT_DURATION * (1 - OVERLAP)
    segments = []
    current = 0
    while current + SEGMENT_DURATION <= duration:
        segments.append((current, current + SEGMENT_DURATION))
        current += step

    if not segments:
        return None, None, None, []

    # 2. 비디오 프레임 추출 (process_video_fast 로직 간소화)
    video_tensors = []
    
    # 프레임 캐싱을 위해 전체를 읽거나, 필요한 것만 읽습니다. 
    # 여기서는 간단히 세그먼트별로 처리 (속도를 위해 최적화 가능)
    for start_sec, end_sec in tqdm(segments, desc="Processing Segments"):
        start_frame = int(start_sec * fps)
        end_frame = int(end_sec * fps)
        
        # 프레임 인덱스 계산 (16개 균등 추출)
        seg_len = end_frame - start_frame
        indices = np.linspace(start_frame, end_frame-1, num=CLIP_LEN, dtype=int)
        
        frames = []
        for idx in indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, RESIZE_HW)
                frames.append(frame)
            else:
                # 실패시 검은 화면
                frames.append(np.zeros((RESIZE_HW[0], RESIZE_HW[1], 3), dtype=np.uint8))
        
        # 정규화 (0~1 -> -1~1) 및 차원 변경 (C, T, H, W)
        frames = np.stack(frames).astype(np.float32) / 255.0
        frames = np.transpose(frames, (3, 0, 1, 2)) # (C, T, H, W)
        frames = (frames - 0.5) / 0.5
        video_tensors.append(torch.from_numpy(frames))
        
    cap.release()

    # 3. 오디오 전처리 (Global Scaling 적용)
    audio_extractor = AudioFeatureExtractorInferenceStyle(AUDIO_CONFIG)
    audio_tensors = []
    
    try:
        y_full, sr = librosa.load(video_path, sr=AUDIO_CONFIG['sample_rate'])
        full_feat = audio_extractor.process_full_audio(y_full)
        
        frames_per_sec = sr / AUDIO_CONFIG['hop_length']
        
        for start_sec, end_sec in segments:
            start_idx = int(start_sec * frames_per_sec)
            end_idx = start_idx + AUDIO_CONFIG['window_size']
            
            feat_seg = full_feat[start_idx:end_idx]
            
            # 패딩
            if feat_seg.shape[0] < AUDIO_CONFIG['window_size']:
                pad = AUDIO_CONFIG['window_size'] - feat_seg.shape[0]
                feat_seg = np.pad(feat_seg, ((0, pad), (0, 0)), mode='constant')
            
            feat_seg = feat_seg[:AUDIO_CONFIG['window_size']]
            audio_tensors.append(torch.from_numpy(feat_seg).float())
            
    except Exception as e:
        print(f"Audio Error: {e}")
        audio_tensors = [torch.zeros(AUDIO_CONFIG['window_size'], AUDIO_CONFIG['n_mels'])] * len(segments)
# ---------------------------------------------------------
    # 4. 텍스트 처리 (List 타입 에러 수정 버전)
    # ---------------------------------------------------------
    text_tensor_path = "/home/stu/ai_project/시연/total_tensor.pt"  # 경로 확인
    
    text_tensors = []
    
    if os.path.exists(text_tensor_path):
        print(f"ℹ️ 텍스트 텐서 로드 중: {text_tensor_path}")
        try:
            # 1. 파일 로드
            loaded_data = torch.load(text_tensor_path, map_location='cpu')
            
            # [디버깅용 출력] 데이터 타입 확인
            # print(f"DEBUG: Loaded data type: {type(loaded_data)}")
            
            # 2. 리스트인 경우 텐서로 변환 (Stack)
            if isinstance(loaded_data, list):
                if len(loaded_data) > 0:
                    # 리스트 안의 텐서들을 하나로 합침
                    # 예: [Tensor(1024), Tensor(1024)] -> Tensor(2, 1024)
                    loaded_text = torch.stack(loaded_data)
                else:
                    # 빈 리스트면 0으로 채움
                    loaded_text = torch.zeros(1024)
            else:
                # 리스트가 아니라 이미 텐서라면 그대로 사용
                loaded_text = loaded_data

            # 3. 차원 축소 (평균)
            # 이제 loaded_text는 무조건 Tensor입니다.
            loaded_text = loaded_text.float()
            
            # (N, 1024) 또는 (N, 1, 1024) 형태일 수 있으므로 평균을 내서 (1024,)로 만듦
            if loaded_text.dim() > 1:
                # 차원이 (N, 1024) 등이면 0번 차원(문장 개수)에 대해 평균
                loaded_text = loaded_text.mean(dim=0)
            
            # 혹시 (1, 1024) 처럼 남아있으면 Squeeze
            if loaded_text.dim() == 2 and loaded_text.shape[0] == 1:
                 loaded_text = loaded_text.squeeze(0)

            # 4. 최종 형태 확인 및 적용
            if loaded_text.shape[0] != 1024:
                print(f"⚠️ 텍스트 차원 이상 ({loaded_text.shape}). 0으로 초기화합니다.")
                loaded_text = torch.zeros(1024)

            # 모든 세그먼트에 동일한 텍스트 피처 적용
            for _ in segments:
                text_tensors.append(loaded_text)
                
        except Exception as e:
            print(f"❌ 텍스트 텐서 로드 실패: {e}")
            print("⚠️ Zero Tensor로 대체합니다.")
            text_tensors = [torch.zeros(1024) for _ in segments]
            
    else:
        print(f"⚠️ 텍스트 파일이 없습니다: {text_tensor_path}")
        print("⚠️ Zero Tensor로 대체합니다.")
        text_tensors = [torch.zeros(1024) for _ in segments]

    return torch.stack(video_tensors), torch.stack(audio_tensors), torch.stack(text_tensors), segments

# =========================================================
# 4. 추론 실행 함수
# =========================================================

import json

@torch.no_grad()
def inference_fusion(video_path):
    print(f"\n🚀 추론 시작: {os.path.basename(video_path)}")
    
    # 1. 데이터 전처리
    v_data, a_data, t_data, segments = preprocess_new_video(video_path)
    
    if v_data is None:
        print("❌ 데이터 처리 실패")
        return

    # 2. 모델 로드
    g_model, a_model, t_model, f_model = load_all_models()
    
    # 3. 배치 단위 추론
    batch_size = 32
    num_samples = len(v_data)
    results = []
    
    print(f"📊 총 세그먼트 수: {num_samples}")
    
    # 배치 처리
    for i in range(0, num_samples, batch_size):
        v_batch = v_data[i:i+batch_size].to(DEVICE)
        a_batch = a_data[i:i+batch_size].to(DEVICE)
        t_batch = t_data[i:i+batch_size].to(DEVICE)
        
        _, g_feats = g_model(v_batch, return_feature=True)
        _, a_feats = a_model(a_batch, return_feature=True)
        
        if hasattr(t_model, 'forward'):
             _, t_feats = t_model(t_batch, return_feature=True)
        else:
             t_feats = t_batch 

        logits = f_model(g_feats, a_feats, t_feats)
        probs = torch.sigmoid(logits).cpu().numpy().flatten()
        
        # 결과 리스트에 담기
        for j, prob in enumerate(probs):
            idx = i + j
            start, end = segments[idx]
            
            # 'label' 키 없이 'score'만 저장합니다.
            results.append({
                "start": round(start, 2),
                "end": round(end, 2),
                "score": float(prob)
            })
            
    # 4. 결과 저장 (JSON)
    save_path = video_path.replace(".mp4", "_results.json")
    # 확장자 대소문자 문제 방지 (.MP4 등)
    if save_path == video_path: save_path = video_path + "_results.json"
        
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=4)
    print(f"\n💾 예측 결과(점수)가 저장되었습니다: {save_path}")

    # 5. 결과 화면 출력 (수정된 부분)
    print("\n[🎯 강조 구간 예측 결과 (임계값 0.5 기준)]")
    
    # 수정 핵심: 'label' 키 대신 'score' 점수를 확인합니다.
    emphasized_segments = [r for r in results if r['score'] >= 0.5]
    
    for res in emphasized_segments:
        print(f"⏰ {res['start']:.2f}초 ~ {res['end']:.2f}초 | 점수: {res['score']:.4f}")
        
    print(f"\n총 {len(emphasized_segments)}/{len(results)} 구간 강조됨.")

# =========================================================
# 실행
# =========================================================
if __name__ == "__main__":
    # 추론할 새로운 영상 경로
    new_video = "/home/stu/ai_project/시연/20251215_195125.mp4" 
    
    if os.path.exists(new_video):
        inference_fusion(new_video)
    else:
        print(f"파일을 찾을 수 없습니다: {new_video}")


🚀 추론 시작: 20251215_195125.mp4


Processing Segments: 100%|██████████| 303/303 [30:30<00:00,  6.04s/it]
/tmp/ipykernel_4154608/3190385592.py:234: UserWarning: PySoundFile failed. Trying audioread instead.
  y_full, sr = librosa.load(video_path, sr=AUDIO_CONFIG['sample_rate'])
/home/stu/anaconda3/envs/ai_project_env/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


ℹ️ 텍스트 텐서 로드 중: /home/stu/ai_project/시연/total_tensor.pt
🔧 모델 로드 중...
  ✅ Gesture Model 로드 완료: /home/stu/ai_project/모델집합/gesture_model.pt
  ✅ Audio Model 로드 완료: /home/stu/ai_project/모델집합/best_bi_lstm.pth
  ℹ️ Text Model: klue/roberta-large (Dim: 1024)
✅ Fusion 모델 가중치 로드 완료
📊 총 세그먼트 수: 303

💾 예측 결과(점수)가 저장되었습니다: /home/stu/ai_project/시연/20251215_195125_results.json

[🎯 강조 구간 예측 결과 (임계값 0.5 기준)]
⏰ 2.50초 ~ 3.50초 | 점수: 0.8511
⏰ 3.50초 ~ 4.50초 | 점수: 0.6189
⏰ 5.00초 ~ 6.00초 | 점수: 0.8406
⏰ 6.00초 ~ 7.00초 | 점수: 0.8173
⏰ 7.00초 ~ 8.00초 | 점수: 0.8308
⏰ 9.00초 ~ 10.00초 | 점수: 0.7637
⏰ 11.00초 ~ 12.00초 | 점수: 0.8489
⏰ 12.50초 ~ 13.50초 | 점수: 0.8030
⏰ 16.00초 ~ 17.00초 | 점수: 0.9051
⏰ 19.50초 ~ 20.50초 | 점수: 0.7852
⏰ 22.50초 ~ 23.50초 | 점수: 0.8407
⏰ 24.00초 ~ 25.00초 | 점수: 0.8473
⏰ 27.50초 ~ 28.50초 | 점수: 0.7552
⏰ 29.00초 ~ 30.00초 | 점수: 1.0000
⏰ 29.50초 ~ 30.50초 | 점수: 0.9979
⏰ 30.00초 ~ 31.00초 | 점수: 0.7849
⏰ 40.00초 ~ 41.00초 | 점수: 0.5301
⏰ 40.50초 ~ 41.50초 | 점수: 0.8126
⏰ 43.00초 ~ 44.00초 | 점수: 0.8506
⏰ 44.00초 ~ 45.00초 | 점수: 0.7